# 下载数据，币安

In [2]:
import pandas as pd
from pymongo import MongoClient
import ccxt

In [3]:
client = MongoClient('mongodb://localhost:27017/')
db = client['crypto']  # 选择数据库
collection = db['ohlcv']  # 选择集合
collection.create_index([("date", 1), ("symbol", 1)])

In [4]:
# =====创建ccxt交易所
p = { 'http': '127.0.0.1:7890', 'https': '127.0.0.1:7890', } # 修改自己的代理端口号即可（具体操作可以查看必看！b圈量化训练营day2新代码操作讲解.PDF）
exchange = ccxt.binance({'proxies': p}) 

# =====获取交易所支持的交易对
markets = exchange.load_markets()
symbols = list(markets.keys())
# end with USDT
symbols = [symbol for symbol in symbols if symbol.endswith('USDT')]
print(symbols)

['BTC/USDT', 'ETH/USDT', 'BNB/USDT', 'BCC/USDT', 'NEO/USDT', 'LTC/USDT', 'QTUM/USDT', 'ADA/USDT', 'XRP/USDT', 'EOS/USDT', 'TUSD/USDT', 'IOTA/USDT', 'XLM/USDT', 'ONT/USDT', 'TRX/USDT', 'ETC/USDT', 'ICX/USDT', 'VEN/USDT', 'NULS/USDT', 'VET/USDT', 'PAX/USDT', 'BCHABC/USDT', 'BSV/USDT', 'USDC/USDT', 'LINK/USDT', 'WAVES/USDT', 'BTT/USDT', 'USDS/USDT', 'ONG/USDT', 'HOT/USDT', 'ZIL/USDT', 'ZRX/USDT', 'FET/USDT', 'BAT/USDT', 'XMR/USDT', 'ZEC/USDT', 'IOST/USDT', 'CELR/USDT', 'DASH/USDT', 'NANO/USDT', 'OMG/USDT', 'THETA/USDT', 'ENJ/USDT', 'MITH/USDT', 'MATIC/USDT', 'ATOM/USDT', 'TFUEL/USDT', 'ONE/USDT', 'FTM/USDT', 'ALGO/USDT', 'USDSB/USDT', 'GTO/USDT', 'ERD/USDT', 'DOGE/USDT', 'DUSK/USDT', 'ANKR/USDT', 'WIN/USDT', 'COS/USDT', 'NPXS/USDT', 'COCOS/USDT', 'MTL/USDT', 'TOMO/USDT', 'PERL/USDT', 'DENT/USDT', 'MFT/USDT', 'KEY/USDT', 'STORM/USDT', 'DOCK/USDT', 'WAN/USDT', 'FUN/USDT', 'CVC/USDT', 'CHZ/USDT', 'BAND/USDT', 'BUSD/USDT', 'BEAM/USDT', 'XTZ/USDT', 'REN/USDT', 'RVN/USDT', 'HC/USDT', 'HBAR/USDT

In [9]:
# str date to time stamp
def str2timestamp(str_date):
    return int(pd.to_datetime(str_date).timestamp() * 1000)

# time stamp to str date
def timestamp2str(timestamp):
    return pd.to_datetime(timestamp, unit='ms').strftime('%Y-%m-%d')

In [12]:
# param int [since]: timestamp in ms of the earliest candle to fetch
# today
data = []
start_tp = str2timestamp('2024-04-01 00:00:00')
end_tp = str2timestamp('2024-04-01 23:59:59')
coin_key = symbols[0]
while start_tp < end_tp:
    price_data = exchange.fetch_ohlcv(timeframe='1s', symbol=coin_key, limit=1000, since=start_tp)  # 获取数据
    price_data = pd.DataFrame(price_data, dtype=float)  # 将数据转换为dataframe
    price_data.rename(columns={0: 'MTS', 1: 'open', 2: 'high',
                    3: 'low', 4: 'close', 5: 'volume'}, inplace=True)  # 重命名
    price_data['candle_begin_time'] = pd.to_datetime(price_data['MTS'], unit='ms')  # 整理时间
    # to json 
    data.extend(price_data.to_dict(orient='records'))
    start_tp = int(data[-1]['MTS'])



In [13]:
# write to mongodb
document = {
    'date' : timestamp2str(data[0]['MTS']),
    'symbol': coin_key ,
    'data': data
}
# insert data
collection.insert_one(document)

InsertOneResult(ObjectId('660b9e1cb38f266ed17641b3'), acknowledged=True)

In [14]:
# 查看data的内存占用大小
print('data size:', pd.DataFrame(data).memory_usage(index=False).sum() / 1024**2, 'MB')

data size: 4.64630126953125 MB


In [21]:
# 根据date和symbol查询数据
cursor = collection.find_one({'date': '2024-04-01', 'symbol': 'BTC/USDT'})
# print(cursor)
cursor

{'_id': ObjectId('660b9e1cb38f266ed17641b3'),
 'date': '2024-04-01',
 'symbol': 'BTC/USDT',
 'data': [{'MTS': 1711929600000.0,
   'open': 71280.0,
   'high': 71280.01,
   'low': 71280.0,
   'close': 71280.0,
   'volume': 0.58399,
   'candle_begin_time': datetime.datetime(2024, 4, 1, 0, 0)},
  {'MTS': 1711929601000.0,
   'open': 71280.01,
   'high': 71288.23,
   'low': 71280.01,
   'close': 71281.29,
   'volume': 8.49194,
   'candle_begin_time': datetime.datetime(2024, 4, 1, 0, 0, 1)},
  {'MTS': 1711929602000.0,
   'open': 71281.29,
   'high': 71281.29,
   'low': 71268.06,
   'close': 71268.06,
   'volume': 6.15201,
   'candle_begin_time': datetime.datetime(2024, 4, 1, 0, 0, 2)},
  {'MTS': 1711929603000.0,
   'open': 71268.06,
   'high': 71268.06,
   'low': 71250.0,
   'close': 71250.0,
   'volume': 5.44813,
   'candle_begin_time': datetime.datetime(2024, 4, 1, 0, 0, 3)},
  {'MTS': 1711929604000.0,
   'open': 71250.01,
   'high': 71250.01,
   'low': 71250.0,
   'close': 71250.01,
   'vo